# Final Project Heart Attack Classification by PatternSeeker

## Import Library

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression # langsung pilih satu algoritma klasifikasi saja
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import pickle

import warnings
warnings.filterwarnings('ignore')

## Import Data

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/Pattern-Seeker/heart-attack-classification/refs/heads/main/heart_attack_prediction_dataset.csv')
df

,Patient ID,Age,Sex,Cholesterol,Blood Pressure,Heart Rate,Diabetes,Family History,Smoking,Obesity,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk
0,BMW7812,67,Male,208,158/88,72,0,0,1,0,...,6.615001,261404,31.251233,286,0,6,Argentina,South America,Southern Hemisphere,0
1,CZE1114,21,Male,389,165/93,98,1,1,1,1,...,4.963459,285768,27.194973,235,1,7,Canada,North America,Northern Hemisphere,0
2,BNI9906,21,Female,324,174/99,72,1,0,0,0,...,9.463426,235282,28.176571,587,4,4,France,Europe,Northern Hemisphere,0
3,JLN3497,84,Male,383,163/100,73,1,1,1,0,...,7.648981,125640,36.464704,378,3,4,Canada,North America,Northern Hemisphere,0
4,GFO8847,66,Male,318,91/88,93,1,1,1,1,...,1.514821,160555,21.809144,231,1,5,Thailand,Asia,Northern Hemisphere,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8758,MSV9918,60,Male,121,94/76,61,1,1,1,0,...,10.806373,235420,19.655895,67,7,7,Thailand,Asia,Northern Hemisphere,0
8759,QSV6764,28,Female,120,157/102,73,1,0,0,1,...,3.833038,217881,23.993866,617,4,9,Canada,North America,Northern Hemisphere,0
8760,XKA5925,47,Male,250,161/75,105,0,1,1,1,...,2.375214,36998,35.406146,527,4,4,Brazil,South America,Southern Hemisphere,1
8761,EPE6801,36,Male,178,119/67,60,1,0,1,0,...,0.029104,209943,27.294020,114,2,8,Brazil,South America,Southern Hemisphere,0


## 1.&nbsp;Data Preprocessing

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8763 entries, 0 to 8762
Data columns (total 26 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Patient ID                       8763 non-null   object 
 1   Age                              8763 non-null   int64  
 2   Sex                              8763 non-null   object 
 3   Cholesterol                      8763 non-null   int64  
 4   Blood Pressure                   8763 non-null   object 
 5   Heart Rate                       8763 non-null   int64  
 6   Diabetes                         8763 non-null   int64  
 7   Family History                   8763 non-null   int64  
 8   Smoking                          8763 non-null   int64  
 9   Obesity                          8763 non-null   int64  
 10  Alcohol Consumption              8763 non-null   int64  
 11  Exercise Hours Per Week          8763 non-null   float64
 12  Diet                

### a.&nbsp;Drop Null

In [5]:
df = df.dropna() # hapus semua baris yg ada null-nya
df

,Patient ID,Age,Sex,Cholesterol,Blood Pressure,Heart Rate,Diabetes,Family History,Smoking,Obesity,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk
0,BMW7812,67,Male,208,158/88,72,0,0,1,0,...,6.615001,261404,31.251233,286,0,6,Argentina,South America,Southern Hemisphere,0
1,CZE1114,21,Male,389,165/93,98,1,1,1,1,...,4.963459,285768,27.194973,235,1,7,Canada,North America,Northern Hemisphere,0
2,BNI9906,21,Female,324,174/99,72,1,0,0,0,...,9.463426,235282,28.176571,587,4,4,France,Europe,Northern Hemisphere,0
3,JLN3497,84,Male,383,163/100,73,1,1,1,0,...,7.648981,125640,36.464704,378,3,4,Canada,North America,Northern Hemisphere,0
4,GFO8847,66,Male,318,91/88,93,1,1,1,1,...,1.514821,160555,21.809144,231,1,5,Thailand,Asia,Northern Hemisphere,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8758,MSV9918,60,Male,121,94/76,61,1,1,1,0,...,10.806373,235420,19.655895,67,7,7,Thailand,Asia,Northern Hemisphere,0
8759,QSV6764,28,Female,120,157/102,73,1,0,0,1,...,3.833038,217881,23.993866,617,4,9,Canada,North America,Northern Hemisphere,0
8760,XKA5925,47,Male,250,161/75,105,0,1,1,1,...,2.375214,36998,35.406146,527,4,4,Brazil,South America,Southern Hemisphere,1
8761,EPE6801,36,Male,178,119/67,60,1,0,1,0,...,0.029104,209943,27.294020,114,2,8,Brazil,South America,Southern Hemisphere,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8763 entries, 0 to 8762
Data columns (total 26 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Patient ID                       8763 non-null   object 
 1   Age                              8763 non-null   int64  
 2   Sex                              8763 non-null   object 
 3   Cholesterol                      8763 non-null   int64  
 4   Blood Pressure                   8763 non-null   object 
 5   Heart Rate                       8763 non-null   int64  
 6   Diabetes                         8763 non-null   int64  
 7   Family History                   8763 non-null   int64  
 8   Smoking                          8763 non-null   int64  
 9   Obesity                          8763 non-null   int64  
 10  Alcohol Consumption              8763 non-null   int64  
 11  Exercise Hours Per Week          8763 non-null   float64
 12  Diet                

In [6]:
df['department'].value_counts()

,count
department,
Sales & Marketing,13711
Operations,9259
Procurement,6639
Technology,6502
Analytics,4610
HR,2060
Finance,1941
R&D,893
Legal,765


In [7]:
df['region'].value_counts()

,count
region,
region_2,10288
region_22,5171
region_7,4200
region_15,2373
region_13,2355
region_26,1867
region_31,1617
region_4,1541
region_27,1355


In [8]:
df['gender'].value_counts()

,count
gender,
m,32347
f,14033


In [9]:
df['education'].value_counts()

,count
education,
Bachelor's,31767
Master's & above,14194
Below Secondary,419


In [10]:
df['recruitment_channel'].value_counts()

,count
recruitment_channel,
other,25747
sourcing,19655
referred,978


In [11]:
df['is_promoted'].value_counts() # imbalanced dataset, ada kelas mayoritas (is_promoted = 0), ada kelas minoritas (is_promoted = 1)

,count
is_promoted,
0,42309
1,4071


In [12]:
df_cp = df.copy()

In [13]:
columns_cat = df_cp.select_dtypes(include = ['object']).columns.to_list()
columns_cat

['department', 'region', 'education', 'gender', 'recruitment_channel']

### b.&nbsp;Encode Categorical Data (Label Encoding/One-Hot Encoding)

In [14]:
label_encoders = {} # dictionary untuk menyimpan objek encoder untuk setiap kolom kategorikal

for col in columns_cat:
  label_encoder = LabelEncoder() # membuat objek label encoder BARU
  label_encoder.fit(df_cp[col])
  df_cp[col] = label_encoder.transform(df_cp[col])
  label_encoders[col] = label_encoder
  pickle.dump(label_encoder, open('label_encoder_'+col+'.pkl', 'wb'))

## 2.&nbsp;Exploratory Data Analysis (EDA)

## 3.&nbsp;Feature Selection

In [15]:
X = df_cp.drop(columns = ['employee_id', 'is_promoted'])
y = df_cp['is_promoted']

In [16]:
X = X.reset_index(drop = True) # untuk mengurutkan kembali indeksnya
y = y.reset_index(drop = True)

## 4.&nbsp;Split Dataset

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 32)

## 5.&nbsp;Model Selection

In [18]:
logreg = LogisticRegression() # instansiasi objek LogisticRegression

## 6.&nbsp;Model Training

In [19]:
model_logreg = logreg.fit(X_train, y_train) #training, setelah selesai, kita dapatkan modelnya yaitu model_logreg

## 7.&nbsp;Model Evaluation

In [20]:
y_pred_train = model_logreg.predict(X_train)
y_pred_test = model_logreg.predict(X_test)

In [21]:
print('training accuracy: ', accuracy_score(y_train, y_pred_train))
print('testing accuracy: ', accuracy_score(y_test, y_pred_test))

training accuracy:  0.9148609314359638
testing accuracy:  0.9151573954290643


## 8.&nbsp;Model Deployment

In [22]:
pickle.dump(model_logreg, open('model_logreg.pkl', 'wb'))

In [23]:
!pip install Flask==3.0.0 pyngrok==7.1.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.2
    Uninstalling Flask-3.1.2:
      Successfully uninstalled Flask-3.1.2


In [24]:
from pyngrok import ngrok
from flask import Flask, jsonify, request
from json import JSONEncoder

In [25]:
ngrok_key = '' # gunakan token masing-masing dari ngrok. jangan gunakan token ini
port = 4999

### a.&nbsp;Deployment URL

In [30]:
ngrok.set_auth_token(ngrok_key)
ngrok.connect(port).public_url

'https://b1c13987a803.ngrok-free.app'

### b.&nbsp;Setup API & Run

In [31]:
# membuat API dan menjalankan web server sederhana dengan framework Flask

app = Flask(__name__)

# fungsi homepage, mengembalikan json
@app.route('/', methods = ['GET'])
def index(): # merepresentasikan satu page
  return {"pesan":"ini halaman homepage"}

@app.route('/predict', methods = ['GET'])
def predict():
  data = request.json
  df_new = pd.DataFrame(data, index = [0])

  # load encoder yg dipakai untuk pre-processing. JANGAN BUAT ENCODER BARU
  with open('/content/label_encoder_department.pkl', 'rb') as file:
    label_encoder_department = pickle.load(file)

  with open('/content/label_encoder_education.pkl', 'rb') as file:
    label_encoder_education = pickle.load(file)

  with open('/content/label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

  with open('/content/label_encoder_recruitment_channel.pkl', 'rb') as file:
    label_encoder_recruitment_channel = pickle.load(file)

  with open('/content/label_encoder_region.pkl', 'rb') as file:
    label_encoder_region = pickle.load(file)

  # encode data baru dengan encoder2 di atas
  df_new['department'] = label_encoder_department.transform(df_new['department']) # JANGAN pakai fungsi FIT_TRANSFORM()
  df_new['education'] = label_encoder_education.transform(df_new['education'])
  df_new['gender'] = label_encoder_gender.transform(df_new['gender'])
  df_new['recruitment_channel'] = label_encoder_recruitment_channel.transform(df_new['recruitment_channel'])
  df_new['region'] = label_encoder_region.transform(df_new['region'])

  # load model untuk prediksi
  with open('/content/model_logreg.pkl', 'rb') as file:
    model_logreg = pickle.load(file)

  y_pred = model_logreg.predict(df_new)
  y_pred_proba = model_logreg.predict_proba(df_new)
  y_pred_proba = y_pred_proba * 100

  return jsonify({"status_prediksi":"berhasil",
                  "is_promoted":y_pred.tolist(),
                  "probability":y_pred_proba.tolist()})


if __name__ == '__main__':
  app.run(port = port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:4999
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [29/Dec/2025 13:38:24] "GET /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Dec/2025 13:38:32] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Dec/2025 13:40:29] "POST / HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [29/Dec/2025 13:41:01] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Dec/2025 13:42:05] "GET /predict HTTP/1.1" 200 -
ERROR:__main__:Exception on /predict [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 1455, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 869, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^